# Qual é o melhor plano?

Você trabalha como analista para a empresa de telecomunicações Megaline. A empresa oferece aos seus cliente planos pré-pagos, Surf e Ultimate. O departamento comercial quer saber quais dos planos dão mais receita para ajustar o orçamento de publicidade.

Você vai realizar uma primeira análise dos planos baseados em uma pequena seleção de clientes. Você terá dados de 500 clientes da Megaline: que clientes são, de onde eles são, qual plano usam, o número de chamadas que eles fizeram e mensagens que eles enviaram em 2018. O seu trabalho é analisar o comportamento dos clientes e determinar quais planos pré-pagos dão mais receita.

Nesse estudo iremos analisar dados de uma empresa de telecomunicações chamada Megaline que oferece planos pré-pagos !
Com isso iremos analisar os planos e a utilização dos planos com os seus clientes ! 

Para este estudo temos as seguintes questões a serem resolvidas:

Realizar o teste de hipótese em tipo de plano e região para saber qual o plano comercial que mais dá receita para a empresa?

- Como é o comportamento dos clientes em relação ao consumo de seus planos?

- As seguintes ações serão realizadas para este caso de estudo

- Ajuste, tratativa e/ou apuramento de valores ausentes e seus tipos de dados

- Junções de DataFrames

- Visualização de gráficos

 -Sugestões para os negócios baseado em dados e hipóteses verificadas

## Inicialização

In [53]:
# Carregando todas as bibliotecas
import pandas as pd
import numpy as np
from scipy import stats as st
import plotly.graph_objects as go
import plotly.express as px

## Carregar dados

In [54]:
# Carregue os arquivos de dados em diferentes DataFrames
try:
    df_call = pd.read_csv('/datasets/Projeto4/megaline_calls.csv')
    df_internet = pd.read_csv('/datasets/Projeto4/megaline_internet.csv')
    df_messages = pd.read_csv('/datasets/Projeto4/megaline_messages.csv')
    df_plans = pd.read_csv('/datasets/Projeto4/megaline_plans.csv')
    df_users = pd.read_csv('/datasets/Projeto4/megaline_users.csv')
except:
    df_call = pd.read_csv('megaline_calls.csv')
    df_internet = pd.read_csv('megaline_internet.csv')
    df_messages = pd.read_csv('megaline_messages.csv')
    df_plans = pd.read_csv('megaline_plans.csv')
    df_users = pd.read_csv('megaline_users.csv')

Abaixo segue as declarações das váriaveis com os devidosdados carregados para a iniciliazação da preparação de análise exploratória dos dados.

## Preparar os dados

[Os dados para este projeto são divididos em várias tabelas. Explore cada um para obter uma compreensão inicial dos dados. Faça as correções necessárias em cada tabela, se necessário.]

## Planos

A tabela plans(dados sobre os planos):

plan_name — o nome do plano de chamadas

usd_monthly_fee — preço mensal em dólares dos EUA

minutes_included — pacote de minutos mensal

messages_included — pacote de mensagens de texto mensal

mb_per_month_included — pacote de volume de dados (em megabytes)

usd_per_minute — preço por minuto depois de exceder o limite do pacote (por exemplo, se o pacote inclui 100 minutos, o 

primeiro minuto excedente será cobrado)


usd_per_message — preço por mensagem de texto depois de exceder o limite do pacote

usd_per_gb — preço por gigabyte extra de dados após exceder o limite do pacote (1 GB = 1024 megabytes)


In [55]:
# Imprima as informações gerais/resumidas sobre o DataFrame dos planos
df_plans.head()

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


In [56]:
# Imprimir uma amostra de dados para os planos
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes


[Descreva o que você vê e observa nas informações gerais e na amostra de dados impressa para o preço dos dados acima. Existem problemas (tipos de dados inadequados, dados ausentes, etc.) que possam precisar de mais investigação e alterações? Como isso pode ser corrigido?]

Há dados com o Dtype errado e com isso logo abaixo reajustarei os dados e os planos comercializados são Surf e Ultimate, sendo que a empresa ajusta segundos em minutos para chamadas, megabytes em gigabytes para dados, em ambos a realização do arredondamento é para cima.

## Corrigir dados

[Corrija problemas óbvios com os dados de acordo com as observações iniciais.]

In [57]:
df_plans['usd_monthly_pay'] = df_plans['usd_monthly_pay'].astype(float)
df_plans['usd_per_gb'] = df_plans['usd_per_gb'].astype(float)
df_plans['mb_per_month_included'] = df_plans['mb_per_month_included'].astype(float)

In [58]:
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      float64
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      float64
 4   usd_per_gb             2 non-null      float64
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(5), int64(2), object(1)
memory usage: 260.0+ bytes


<div class="alert alert-sucess">
<strong>Comentário do revisor v1</strong>

Bom trabalho!
</div>

## Enriqueça os dados

[Adicione fatores adicionais aos dados se você acredita que eles podem ser úteis.]

In [59]:
# Criei uma coluna a patir de uma coluna que já está presente ,transformando os megabytes por gigabytes
df_plans['gb_per_month_included'] = df_plans['mb_per_month_included'] / 1024

df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      float64
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      float64
 4   usd_per_gb             2 non-null      float64
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
 8   gb_per_month_included  2 non-null      float64
dtypes: float64(6), int64(2), object(1)
memory usage: 276.0+ bytes


Seus planos comercializados são Surf e Ultimate, sendo que a empresa ajusta segundos em minutos para chamadas, megabytes em gigabytes para dados, em ambos a realização do arredondamento é para cima.

## Usuários

A tabela users (dados sobre usuários):

- user_id — identificação do usuário
- first_name — nome do usuário
- last_name — último sobrenome do usuário
- age — idade do usuário (em anos)
- reg_date — data da inscrição (dd, mm, aa)
- churn_date — a data que o usuário parou de usar o serviço (se o valor for ausente, o plano estava sendo usado quando esse dado foi gerado)
- city — cidade de residência do usuário
- plan — nome do plano


In [60]:
# Imprima as informações gerais/resumidas sobre o DataFrame dos usuários
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [61]:
# Imprimir uma amostra de dados para usuários
df_users.head(10)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN
5,1005,Livia,Shields,31,"Dallas-Fort Worth-Arlington, TX MSA",2018-11-29,surf,NaN
6,1006,Jesusa,Bradford,73,"San Francisco-Oakland-Berkeley, CA MSA",2018-11-27,ultimate,2018-12-18
7,1007,Eusebio,Welch,42,"Grand Rapids-Kentwood, MI MSA",2018-07-11,surf,NaN
8,1008,Emely,Hoffman,53,"Orlando-Kissimmee-Sanford, FL MSA",2018-08-03,ultimate,NaN
9,1009,Gerry,Little,19,"San Jose-Sunnyvale-Santa Clara, CA MSA",2018-04-22,surf,NaN


[Descreva o que você vê e observa nas informações gerais e na amostra de dados impressa para o preço dos dados acima. Existem problemas (tipos de dados inadequados, dados ausentes, etc.) que possam precisar de mais investigação e alterações? Como isso pode ser corrigido?]

### Corrigir dados

[Corrija problemas óbvios com os dados de acordo com as observações iniciais.]

In [62]:
df_users['reg_date'] = df_users['reg_date'].astype('datetime64[ns]')

Podemos verificar que esta tabela possui uma variável “churn_date” com dados ausentes, a variável “reg_date está com dado incorreto, precisa ser alterado para formato de data.

In [63]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     500 non-null    int64         
 1   first_name  500 non-null    object        
 2   last_name   500 non-null    object        
 3   age         500 non-null    int64         
 4   city        500 non-null    object        
 5   reg_date    500 non-null    datetime64[ns]
 6   plan        500 non-null    object        
 7   churn_date  34 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 31.4+ KB


### Enriquecer Dados

[Adicione fatores adicionais aos dados se você acredita que eles podem ser úteis.]

Como a maior parte dos valores da coluna churn_date estão com NaN pelo motivo de o usuário não ter parado com o serviço , ou seja, seu plano está ativo ,com isso troquei todos NaN por Ativos

In [64]:
df_users['status_plan'] = df_users['churn_date'].fillna(0)
df_users['status_plan'] = df_users['status_plan'].apply(lambda x: "Desativado" if x != 0 else "Ativo")

In [65]:
df_users.head(10)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,status_plan
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN,Ativo
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN,Ativo
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN,Ativo
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN,Ativo
5,1005,Livia,Shields,31,"Dallas-Fort Worth-Arlington, TX MSA",2018-11-29,surf,NaN,Ativo
6,1006,Jesusa,Bradford,73,"San Francisco-Oakland-Berkeley, CA MSA",2018-11-27,ultimate,2018-12-18,Desativado
7,1007,Eusebio,Welch,42,"Grand Rapids-Kentwood, MI MSA",2018-07-11,surf,NaN,Ativo
8,1008,Emely,Hoffman,53,"Orlando-Kissimmee-Sanford, FL MSA",2018-08-03,ultimate,NaN,Ativo
9,1009,Gerry,Little,19,"San Jose-Sunnyvale-Santa Clara, CA MSA",2018-04-22,surf,NaN,Ativo


In [66]:
df_users['age'].describe()

count    500.000000
mean      45.486000
std       16.972269
min       18.000000
25%       30.000000
50%       46.000000
75%       61.000000
max       75.000000
Name: age, dtype: float64

Pela distribuição da idade dos usuários da empresa, já podemos identificar que possuem clientes muito jovens desde 18 anos até 75 anos de idade, com um desvio padrão alto de 16,97%, podemos levantar a hipótese que diferenças de utilização do plano por faixa etária

## Chamadas

A tabela calls (dados sobre as chamadas)
- id — identificador de chamada unívoco
- call_date — data da chamada
- duration — duração da chamada (em minutos)
- user_id — o identificador do usuário que faz a chamada


In [67]:
# Imprima as informações gerais/resumidas sobre o DataFrame das chamadas
df_call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


In [68]:
# Imprimir uma amostra de dados para chamadas
df_call.head(10)

,id,user_id,call_date,duration
0,1000_93,1000,2018-12-27,8.52
1,1000_145,1000,2018-12-27,13.66
2,1000_247,1000,2018-12-27,14.48
3,1000_309,1000,2018-12-28,5.76
4,1000_380,1000,2018-12-30,4.22
5,1000_388,1000,2018-12-31,2.20
6,1000_510,1000,2018-12-27,5.75
7,1000_521,1000,2018-12-28,14.18
8,1000_530,1000,2018-12-28,5.77
9,1000_544,1000,2018-12-26,4.40


[Descreva o que você vê e observa nas informações gerais e na amostra de dados impressa para o preço dos dados acima. Existem problemas (tipos de dados inadequados, dados ausentes, etc.) que possam precisar de mais investigação e alterações? Como isso pode ser corrigido?]

### Corrigir dados

[Corrija problemas óbvios com os dados de acordo com as observações iniciais.]

In [69]:
# Verificando se a alteração no Dtype deu tudo certo
df_call['call_date'] = df_call['call_date'].astype('datetime64[ns]')
df_call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         137735 non-null  object        
 1   user_id    137735 non-null  int64         
 2   call_date  137735 non-null  datetime64[ns]
 3   duration   137735 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 4.2+ MB


In [70]:
df_call.isna().sum()

id           0
user_id      0
call_date    0
duration     0
dtype: int64

Não possui dados ausentes e nem duplicados, porém verificamos que os dados de “call_date” está, incorreto será alterado para o formato data

Não possui valores ausentes ,nem duplicados e ainda percebemos chamadas com zero minutos que irei desconsiderar

Dentro do dataset, existem uma parte dos dados que possuem chamadas de zero minutos, para estes dados, serão desconsiderados para a análise.

### Enriquecer dados

[Adicione fatores adicionais aos dados se você acredita que eles podem ser úteis.]

In [71]:
df_call['mes'] = df_call['call_date'].dt.month
df_call['ano'] = df_call['call_date'].dt.year
df_call.head()

,id,user_id,call_date,duration,mes,ano
0,1000_93,1000,2018-12-27,8.52,12,2018
1,1000_145,1000,2018-12-27,13.66,12,2018
2,1000_247,1000,2018-12-27,14.48,12,2018
3,1000_309,1000,2018-12-28,5.76,12,2018
4,1000_380,1000,2018-12-30,4.22,12,2018


In [72]:
# Aplicando função de arredondamento para cima
df_call['arredondados'] = df_call['duration'].apply(np.ceil)
df_call.head() 

,id,user_id,call_date,duration,mes,ano,arredondados
0,1000_93,1000,2018-12-27,8.52,12,2018,9.0
1,1000_145,1000,2018-12-27,13.66,12,2018,14.0
2,1000_247,1000,2018-12-27,14.48,12,2018,15.0
3,1000_309,1000,2018-12-28,5.76,12,2018,6.0
4,1000_380,1000,2018-12-30,4.22,12,2018,5.0


In [73]:
df_call_called = df_call.pivot_table(index=['user_id','mes'], values='arredondados', aggfunc=('mean','sum')).reset_index()
df_call_called.columns = ['user_id', 'mes', 'consumo_mes','qdt_chamadas']

In [74]:
df_call_called

,user_id,mes,consumo_mes,qdt_chamadas
0,1000,12,7.750000,124.0
1,1001,8,6.740741,182.0
2,1001,9,6.428571,315.0
3,1001,10,6.046154,393.0
4,1001,11,6.656250,426.0
...,...,...,...,...
2253,1498,12,8.692308,339.0
2254,1499,9,8.439024,346.0
2255,1499,10,7.264151,385.0
2256,1499,11,6.844444,308.0


In [75]:
df_sem_zero_call = df_call[df_call['duration'] != 0]
df_sem_zero_call.pivot_table(index='user_id',values='arredondados',aggfunc='mean')

,arredondados
user_id,
1000,7.750000
1001,8.554455
1002,8.819149
1003,9.355932
1004,9.058824
...,...
1495,8.485577
1496,9.208861
1497,7.142857


In [76]:
df_sem_zero_call['duration'].describe()

count    110901.000000
mean          8.378196
std           5.354567
min           0.010000
25%           4.150000
50%           7.710000
75%          11.820000
max          37.600000
Name: duration, dtype: float64

Percebemos que a duração mínima de chamadas é de menos que 1 e a máxima é mais de 37 tendo a média de 8.37 

## Mensagens

A tabela messages (dados nas mensagens de texto):
- id — identificador unívoco de mensagem de textos
- message_date — data da mensagem de texto
- user_id — o identificador do usuário que envia a mensagem de texto


In [77]:
# Imprima as informações gerais/resumidas sobre o DataFrame das mensagens
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [78]:
# Imprima uma amostra dos dados para as mensagens
df_messages.head()

,id,user_id,message_date
0,1000_125,1000,2018-12-27
1,1000_160,1000,2018-12-31
2,1000_223,1000,2018-12-31
3,1000_251,1000,2018-12-27
4,1000_255,1000,2018-12-26


In [79]:
df_messages.isna().sum()

id              0
user_id         0
message_date    0
dtype: int64

Podemos verificar que esta tabela não possui dados ausentes e dados duplicados

[Descreva o que você vê e observa nas informações gerais e na amostra de dados impressa para o preço dos dados acima. Existem problemas (tipos de dados inadequados, dados ausentes, etc.) que possam precisar de mais investigação e alterações? Como isso pode ser corrigido?]

### Corrigir dados

[Corrija problemas óbvios com os dados de acordo com as observações iniciais.]

In [80]:
df_messages['message_date'] = df_messages['message_date'].astype('datetime64[ns]')

In [81]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            76051 non-null  object        
 1   user_id       76051 non-null  int64         
 2   message_date  76051 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ MB


### Enriquecer dados

[Adicione fatores adicionais aos dados se você acredita que eles podem ser úteis.]

In [82]:
df_messages['mes'] = df_messages['message_date'].dt.month
df_messages['ano'] = df_messages['message_date'].dt.year
df_messages.head()

,id,user_id,message_date,mes,ano
0,1000_125,1000,2018-12-27,12,2018
1,1000_160,1000,2018-12-31,12,2018
2,1000_223,1000,2018-12-31,12,2018
3,1000_251,1000,2018-12-27,12,2018
4,1000_255,1000,2018-12-26,12,2018


In [83]:
df_messages_consumo = df_messages.pivot_table(index = ['user_id', 'mes'], values = 'id', aggfunc = 'count')

In [84]:
df_messages_consumo.columns = ['qdt_msg_enviadas']
df_messages_consumo.head() 

qdt_msg_enviadas
user_id mes                  
1000    12                 11
1001    8                  30
        9                  44
        10                 53
        11                 36

## Internet

A tabela internet (dados sobre sessões web):
- id — identificador de sessão unívoco
- mb_used — o volume de dados gasto durante a sessão ( em megabytes)
- session_date — data da sessão web
- user_id — identificador do usuário

In [85]:
# Imprime as informações gerais/resumidas sobre o DataFrame da Internet
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


In [86]:
# Imprime uma amostra de dados para o tráfego da Internet
df_internet.head()

,id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29,89.86
1,1000_204,1000,2018-12-31,0.00
2,1000_379,1000,2018-12-28,660.40
3,1000_413,1000,2018-12-26,270.99
4,1000_442,1000,2018-12-27,880.22


In [87]:
df_internet.duplicated().sum()

0

Podemos verificar que esta tabela não possui dados ausentes e dados duplicados

[Descreva o que você vê e observa nas informações gerais e na amostra de dados impressa para o preço dos dados acima. Existem problemas (tipos de dados inadequados, dados ausentes, etc.) que possam precisar de mais investigação e alterações? Como isso pode ser corrigido?]

### Corrigir dados

[Corrija problemas óbvios com os dados de acordo com as observações iniciais.]


In [88]:
df_internet['session_date'] = df_internet['session_date'].astype('datetime64[ns]')
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            104825 non-null  object        
 1   user_id       104825 non-null  int64         
 2   session_date  104825 non-null  datetime64[ns]
 3   mb_used       104825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.2+ MB


### Enriquecer dados

[Adicione fatores adicionais aos dados se você acredita que eles podem ser úteis.]

In [89]:
df_internet['mes'] = df_internet['session_date'].dt.month
df_internet['year'] = df_internet['session_date'].dt.year
df_internet.head()

,id,user_id,session_date,mb_used,mes,year
0,1000_13,1000,2018-12-29,89.86,12,2018
1,1000_204,1000,2018-12-31,0.00,12,2018
2,1000_379,1000,2018-12-28,660.40,12,2018
3,1000_413,1000,2018-12-26,270.99,12,2018
4,1000_442,1000,2018-12-27,880.22,12,2018


In [90]:
df_internet_cons = df_internet.pivot_table(index = ['user_id', 'mes'], values = 'mb_used', aggfunc = 'sum')

In [91]:
df_internet_cons.columns = ['total_consumo_mb_internet']
df_internet_cons.head(10)

total_consumo_mb_internet
user_id mes                           
1000    12                     1901.47
1001    8                      6919.15
        9                     13314.82
        10                    22330.49
        11                    18504.30
        12                    19369.18
1002    10                     6552.01
        11                    19345.08
        12                    14396.24
1003    12                    27044.14

In [92]:
gb_arround = np.ceil(df_internet_cons/1024)

In [93]:
df_internet_cons['gb_used_arredondado'] = df_internet_cons['total_consumo_mb_internet'].apply(np.ceil)
df_internet_cons.head(2)

,,total_consumo_mb_internet,gb_used_arredondado
user_id,mes,,
1000,12,1901.47,1902.0
1001,8,6919.15,6920.0


Analisando o crescimento da utilização de dados, podemos perceber que o crescimento de clientes dentro da base de usuário cresceu exponencialmente o que colabora para a utilização de dados de internet assim como todos os demais serviços fornecidos.



## Condições do plano de estudos

[É fundamental entender como os planos funcionam, como os usuários são cobrados com base na assinatura do plano. Assim, sugerimos imprimir as informações do plano para visualizar novamente suas condições.]

In [94]:
# Imprime as condições do plano e certifique-se de que estão claras para você
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      float64
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      float64
 4   usd_per_gb             2 non-null      float64
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
 8   gb_per_month_included  2 non-null      float64
dtypes: float64(6), int64(2), object(1)
memory usage: 276.0+ bytes


## Dados agregados por usuário

[Agora, como os dados estão limpos, agregue os dados por usuário por período para ter apenas um registro por usuário por período. Irá facilitar muito a análise posterior.]

In [95]:
df_call_called.head()

,user_id,mes,consumo_mes,qdt_chamadas
0,1000,12,7.750000,124.0
1,1001,8,6.740741,182.0
2,1001,9,6.428571,315.0
3,1001,10,6.046154,393.0
4,1001,11,6.656250,426.0


In [96]:
# Calcule o número de chamadas feitas por cada usuário por mês. Salve o resultado.

df_calls_mes = pd.merge(df_call_called, df_internet_cons, on='user_id', how='outer')
df_calls_mes

,user_id,mes,consumo_mes,qdt_chamadas,total_consumo_mb_internet,gb_used_arredondado
0,1000,12.0,7.750000,124.0,1901.47,1902.0
1,1001,8.0,6.740741,182.0,6919.15,6920.0
2,1001,8.0,6.740741,182.0,13314.82,13315.0
3,1001,8.0,6.740741,182.0,22330.49,22331.0
4,1001,8.0,6.740741,182.0,18504.30,18505.0
...,...,...,...,...,...,...
14664,1204,NaN,NaN,NaN,21346.95,21347.0
14665,1204,NaN,NaN,NaN,36730.05,36731.0
14666,1349,NaN,NaN,NaN,13093.55,13094.0
14667,1349,NaN,NaN,NaN,17128.26,17129.0


In [97]:
# Calcule a quantidade de minutos gastos por cada usuário por mês. Salve o resultado.
df_users_minutes = pd.merge(df_calls_mes, df_messages_consumo, on=['user_id', 'mes'], how='outer')
df_users_minutes

,user_id,mes,consumo_mes,qdt_chamadas,total_consumo_mb_internet,gb_used_arredondado,qdt_msg_enviadas
0,1000,12.0,7.750000,124.0,1901.47,1902.0,11.0
1,1001,8.0,6.740741,182.0,6919.15,6920.0,30.0
2,1001,8.0,6.740741,182.0,13314.82,13315.0,30.0
3,1001,8.0,6.740741,182.0,22330.49,22331.0,30.0
4,1001,8.0,6.740741,182.0,18504.30,18505.0,30.0
...,...,...,...,...,...,...,...
14697,1349,10.0,NaN,NaN,NaN,NaN,76.0
14698,1349,11.0,NaN,NaN,NaN,NaN,72.0
14699,1349,12.0,NaN,NaN,NaN,NaN,61.0
14700,1361,5.0,NaN,NaN,NaN,NaN,2.0


In [98]:
# Calcule o número de mensagens enviadas por cada usuário por mês. Salve o resultado.
df_messages_enviadas = pd.merge(df_users_minutes, df_users, on='user_id', how='left')
df_messages_enviadas

,user_id,mes,consumo_mes,qdt_chamadas,total_consumo_mb_internet,gb_used_arredondado,qdt_msg_enviadas,first_name,last_name,age,city,reg_date,plan,churn_date,status_plan
0,1000,12.0,7.750000,124.0,1901.47,1902.0,11.0,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN,Ativo
1,1001,8.0,6.740741,182.0,6919.15,6920.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
2,1001,8.0,6.740741,182.0,13314.82,13315.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
3,1001,8.0,6.740741,182.0,22330.49,22331.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
4,1001,8.0,6.740741,182.0,18504.30,18505.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14697,1349,10.0,NaN,NaN,NaN,NaN,76.0,Florentina,Diaz,69,"Boston-Cambridge-Newton, MA-NH MSA",2018-10-01,surf,NaN,Ativo
14698,1349,11.0,NaN,NaN,NaN,NaN,72.0,Florentina,Diaz,69,"Boston-Cambridge-Newton, MA-NH MSA",2018-10-01,surf,NaN,Ativo
14699,1349,12.0,NaN,NaN,NaN,NaN,61.0,Florentina,Diaz,69,"Boston-Cambridge-Newton, MA-NH MSA",2018-10-01,surf,NaN,Ativo
14700,1361,5.0,NaN,NaN,NaN,NaN,2.0,Jacelyn,Hoffman,45,"Birmingham-Hoover, AL MSA",2018-04-14,surf,NaN,Ativo


Percebe-se que com a junção dos Dataframes apareceu valores ausentes, ocorrendo pelo fato das chaves não encontrar valores que o correspondem, gerando valores ausentes que serão tratados !

In [99]:
# Calcule o volume de tráfego de internet usado por cada usuário por mês. Salve o resultado.
df_trafego = pd.merge(df_messages_enviadas ,df_users, on='user_id', how='left') 
df_trafego

,user_id,mes,consumo_mes,qdt_chamadas,total_consumo_mb_internet,gb_used_arredondado,qdt_msg_enviadas,first_name_x,last_name_x,age_x,...,churn_date_x,status_plan_x,first_name_y,last_name_y,age_y,city_y,reg_date_y,plan_y,churn_date_y,status_plan_y
0,1000,12.0,7.750000,124.0,1901.47,1902.0,11.0,Anamaria,Bauer,45,...,NaN,Ativo,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN,Ativo
1,1001,8.0,6.740741,182.0,6919.15,6920.0,30.0,Mickey,Wilkerson,28,...,NaN,Ativo,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
2,1001,8.0,6.740741,182.0,13314.82,13315.0,30.0,Mickey,Wilkerson,28,...,NaN,Ativo,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
3,1001,8.0,6.740741,182.0,22330.49,22331.0,30.0,Mickey,Wilkerson,28,...,NaN,Ativo,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
4,1001,8.0,6.740741,182.0,18504.30,18505.0,30.0,Mickey,Wilkerson,28,...,NaN,Ativo,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14697,1349,10.0,NaN,NaN,NaN,NaN,76.0,Florentina,Diaz,69,...,NaN,Ativo,Florentina,Diaz,69,"Boston-Cambridge-Newton, MA-NH MSA",2018-10-01,surf,NaN,Ativo
14698,1349,11.0,NaN,NaN,NaN,NaN,72.0,Florentina,Diaz,69,...,NaN,Ativo,Florentina,Diaz,69,"Boston-Cambridge-Newton, MA-NH MSA",2018-10-01,surf,NaN,Ativo
14699,1349,12.0,NaN,NaN,NaN,NaN,61.0,Florentina,Diaz,69,...,NaN,Ativo,Florentina,Diaz,69,"Boston-Cambridge-Newton, MA-NH MSA",2018-10-01,surf,NaN,Ativo
14700,1361,5.0,NaN,NaN,NaN,NaN,2.0,Jacelyn,Hoffman,45,...,NaN,Ativo,Jacelyn,Hoffman,45,"Birmingham-Hoover, AL MSA",2018-04-14,surf,NaN,Ativo


In [100]:
df_messages_enviadas.isna().sum()

user_id                          0
mes                             30
consumo_mes                     63
qdt_chamadas                    63
total_consumo_mb_internet       41
gb_used_arredondado             41
qdt_msg_enviadas              3491
first_name                       0
last_name                        0
age                              0
city                             0
reg_date                         0
plan                             0
churn_date                   13653
status_plan                      0
dtype: int64

In [101]:
# Tratando os valores ausentes 
df_messages_enviadas['mes'].fillna(0, inplace = True)
df_messages_enviadas['consumo_mes'].fillna(0, inplace = True)
df_messages_enviadas['qdt_chamadas'].fillna(0, inplace = True)
df_messages_enviadas['total_consumo_mb_internet'].fillna(0, inplace = True)
df_messages_enviadas['gb_used_arredondado'].fillna(0, inplace = True)
df_messages_enviadas['qdt_msg_enviadas'].fillna(0, inplace = True)
df_messages_enviadas.head()

,user_id,mes,consumo_mes,qdt_chamadas,total_consumo_mb_internet,gb_used_arredondado,qdt_msg_enviadas,first_name,last_name,age,city,reg_date,plan,churn_date,status_plan
0,1000,12.0,7.750000,124.0,1901.47,1902.0,11.0,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN,Ativo
1,1001,8.0,6.740741,182.0,6919.15,6920.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
2,1001,8.0,6.740741,182.0,13314.82,13315.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
3,1001,8.0,6.740741,182.0,22330.49,22331.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo
4,1001,8.0,6.740741,182.0,18504.30,18505.0,30.0,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN,Ativo


In [102]:
df_messages_enviadas.isna().sum()

user_id                          0
mes                              0
consumo_mes                      0
qdt_chamadas                     0
total_consumo_mb_internet        0
gb_used_arredondado              0
qdt_msg_enviadas                 0
first_name                       0
last_name                        0
age                              0
city                             0
reg_date                         0
plan                             0
churn_date                   13653
status_plan                      0
dtype: int64

[Coloque os dados agregados em um DataFrame para que um registro represente o que um usuário único consumiu em um determinado mês.]

In [107]:
# Junte os dados de chamadas, minutos, mensagens, internet com base em user_id e mês
aux = df_users.merge(df_call, on='user_id', how='left')
aux1 = aux.merge(df_internet, on='user_id', how='inner')
df = aux1.merge(df_messages, how='inner')
df_messages_enviadas.head()

MemoryError: Unable to allocate 649. MiB for an array with shape (2, 42554594) and data type int64

In [ ]:
df_messages_enviadas.info()

Nos blocos de códigos acima temos uma sequência de funções que foram criadas e testadas com o objetivo de agregar informações sobre o comportamento de cada usuário referente a utilização do seu plano, com foco em contabilizar quais usuários, planos e serviços tiveram mais excedentes de utilização acima do plano contratado

[A receita mensal para cada usuário (subtraia o limite do pacote grátis do número total de chamadas, mensagens de texto, e dados; multiplique o resultado pelos valores do plano; adicione o preço mensal dependendo do plano de chamada). N.B. Isso pode não ser tão trivial quanto apenas algumas linhas, dadas as condições do plano! Portanto, não há problema em gastar algum tempo nisso.]

In [ ]:
#Criando função para chamadas excedidas
def exceded_call(row):
    plano = row['plan']
    consumo_mes = row['consumo_mes']
    
    if plano == 'surf':
        if consumo_mes > 500:
            ch_exceeded = consumo_mes - 500
        else:
            ch_exceeded = 0
    if plano == 'ultimate':
        if consumo_mes > 3000:
            ch_exceeded = consumo_mes - 3000
        else:
            ch_exceeded = 0
    return ch_exceeded

In [ ]:
# Aplicar a função e criar a coluna de excedente de chamadas e verificar se ocorreu corretamente
df_messages_enviadas['chamada_excedida'] = df_messages_enviadas.apply(exceded_call, axis = 1)

In [ ]:
# criando função de excedente de mensagens

def exceded_msg(row):
    plano = row['plan']
    msg_enviada = row['qdt_chamadas']
    
    if plano == 'surf':
        if msg_enviada > 50:
            msg_exceeded = msg_enviada - 50
        else:
            msg_exceeded = 0
    if plano == 'ultimate':
        if msg_enviada > 1000:
            msg_exceeded = msg_enviada - 1000
        else:
            msg_exceeded = 0
    return msg_exceeded

In [ ]:
# Aplicar a função e criar a coluna de excedente de mensagens
df_messages_enviadas['msg_excedidas'] = df_messages_enviadas.apply(exceded_msg, axis = 1)

In [ ]:
# Criando função para calcular excedente de utilização do pacote de dados

def exceded_internet(row):
    plano = row['plan']
    gb_used = row['gb_used_arredondado']
    
    if plano == 'surf':
        if gb_used > 15:
            internet_exceeded = gb_used - 15
        else:
            internet_exceeded = 0
    if plano == 'ultimate':
        if gb_used > 30:
            internet_exceeded = gb_used - 30
        else:
            internet_exceeded = 0
    return internet_exceeded

In [ ]:
# Aplicando a função e criando nova coluna no dataframe
df_messages_enviadas['internet_excedidas'] = df_messages_enviadas.apply(exceded_internet, axis = 1)

In [ ]:
# verificando se todas novas colunas foram criadas com sucesso
df_messages_enviadas.head(10)

In [ ]:
# Adicione as informações do plano
df_plans

# PLANO SURF 

plano_surf = 20
qtd_ch_surf = 500
qtd_msg_surf = 50
data_internet_surf = 15
chamadas_excedidas_surf = 0.03
msg_excedidas_surf = 0.03
internet_excedidas_surf = 10

# PLANO ULTIMATE

plano_ultimate = 70
qtd_ch_ultimate = 3000
qdt_msg_ultimate = ultimate = 1000
data_internet_ultimate = 30
chamadas_excedidas_ultimate = 0.01
msg_excedidas_ultimate = 0.01
internet_excedidas_ultimate = 7

In [ ]:
# Calcule a receita mensal de cada usuário
def receita_usuario(row):
    plano = row['plan']
    chamada_excedida = row['chamada_excedida']
    msg_excedida = row['msg_excedidas']
    internet_excedida = row['internet_excedidas']
    
    if plano == 'surf':
        receita_usuario = plano_surf + ((chamada_excedida * chamadas_excedidas_surf) + (msg_excedida * msg_excedidas_surf) + (internet_excedida * internet_excedidas_surf))
    else:
        receita_usuario = plano_ultimate + ((chamada_excedida * chamadas_excedidas_ultimate) + (msg_excedida * msg_excedidas_ultimate) + (internet_excedida * internet_excedidas_ultimate))
    return receita_usuario

In [ ]:
df_messages_enviadas['receita_usuario'] = df_messages_enviadas.apply(receita_usuario, axis = 1)
df_messages_enviadas.head()

## Estudar o comportamento do usuário

[Calcule algumas estatísticas descritivas úteis para os dados agregados e unidos, que geralmente revelam uma imagem geral capturada pelos dados. Desenhe gráficos úteis para ajudar na compreensão. Dado que a tarefa principal é comparar os planos e decidir qual é mais rentável, as estatísticas e as parcelas devem ser calculadas por plano.]

[Existem dicas relevantes nos comentários para Chamadas, mas elas não são fornecidas para Mensagens e Internet, embora o princípio do estudo estatístico seja o mesmo para elas e para Chamadas.]

### Chamadas

In [ ]:
# Compare a duração média das ligações de cada plano para cada mês distinto. Faça um gráfico de barras para visualizá-lo.
df_messages_enviadas.pivot_table(index='plan',columns='mes',values='consumo_mes',aggfunc='mean').plot(kind='bar',figsize=(7,4),alpha = 0.5)

Acima temos uma análise estatística descritiva sobre a média de minutos do serviço de chamadas utilizadas por plano de seus usuários, pois verificamos que as modulações que ocorrem nos dois planos e os picos estão mais presentes nos meses 7 e 5 

In [ ]:
# Compare o número de minutos que os usuários de cada plano requerem a cada mês. Construa um histograma.
df_messages_enviadas.hist('consumo_mes',bins=20)

[Calcule a média e a variável da duração da chamada para raciocinar se os usuários de diferentes planos possuem comportamentos diferentes em suas chamadas.]

In [ ]:
# Calcule a média e a variação da duração mensal das chamadas

y = df_messages_enviadas.groupby('plan')['consumo_mes'].mean()
y.plot(kind='pie',figsize=(6,4), autopct='%1.1f%%', colors=['#03fcf0','#0335fc']);
y

Verificando o gráfico acima que mostra o percentual de utilização de chamadas por plano, temos uma utilização quase igual de chamadas em ambos os planos o que indica que o plano surf mesmo com mais dados de pacote ,mesmo sendo pequena a diferença, tem grande usabilidade de utilização por seus usuários, iremos realizar uma abertura maior dos dados para identificar possíveis oportunidades.

In [ ]:
# Faça um diagrama de caixa para visualizar a distribuição da duração mensal das chamadas

surf = df_messages_enviadas[df_messages_enviadas['plan'] == 'surf']
surf.boxplot('consumo_mes');
surf

In [ ]:
# segmentando dados para verificação de diagrama de caixa para consumo de chamadas por plano

ultimate = df_messages_enviadas[df_messages_enviadas['plan'] == 'ultimate']

# grafico de diagrama de caixa para visualizar a distribuição da duração mensal das chamadas

ultimate.boxplot('consumo_mes');

Percebe-se que há media esta em torno de 7,5 e é perceptível que uma massa de usuários do plano surf apresentam valores de minutos de chamada acima do pacote contratado fazendo com que os mesmo paguem por valores excedentes  já o que não ocorre dentro do plano ultimate demostrando comportamentos diferentes entre os usuário podemos identificar possibilidades comerciais de ofertar uma maior gama de planos com minutos diferentes para capitalizar novos clientes uma vez que já com esta amostragem identificamos estes comportamentos.

[Formule conclusões sobre como os usuários se comportam em termos de chamada. O comportamento deles é diferente entre os planos?]

### Mensagens

In [ ]:
# Compare o número de mensagens que os usuários de cada plano tendem a enviar a cada mês
w = df_messages_enviadas.pivot_table(index='plan',columns='mes',values='reg_date',aggfunc='count')
w.mean().plot(figsize=(10,5),color='#03fcf0',linewidth=2,title='Mensagens Totais Enviadas Mês a Mês',
              xlabel='Mês',ylabel='Quantidade de Mensagens Enviadas');

In [ ]:
# Compare a quantidade de tráfego de internet consumido pelos usuários por plano

df_messages_enviadas.pivot_table(index='plan',columns='mes',values='reg_date',aggfunc='count').plot(kind='bar',figsize=(7,4),alpha=0.5);

In [ ]:
w = df_messages_enviadas.groupby('plan')['reg_date'].count()
w.plot(kind='pie',figsize=(6,4),autopct='%1.1f%%',colors=['#03fcf0','#0335fc']);

O mesmo comportamento dos usuários se repetiram dentro do serviço de utilização de mensagens, assim como das chamadas, podemos que houve uma crescente utilização dos serviço mês a mês e isso se deve a entrada de novos clientes para a base de dados, a quantidade e percentual de utilização deste serviço é maior para os usuários do plano surf, podemos destacar que o percentual de utilização deste serviço de mensagens é maior que o ultimate, o que é congruente pelo fato do plano ter menos minutos de chamadas seu perfil de clientes podem preferir usar mais envio de mensagens é uma possibilidade de maior percentual de utilização por parte deste perfil de usuários

[Formule conclusões sobre como os usuários se comportam em termos de mensagens. O comportamento deles é diferente entre os planos?]

### Internet

In [ ]:
df_messages_enviadas.pivot_table(index='plan',columns='mes',values='gb_used_arredondado',aggfunc='sum').plot(kind='bar',figsize=(7,4),alpha=0.5);


In [ ]:
w = (df_messages_enviadas['gb_used_arredondado'] == 0).sum()
y = df_messages_enviadas['user_id'].unique().shape[0]
(w / y) * 100
print('A empresa possui um percentual de',(w / y) * 100,'clientes sem utilização do serviço de internet.')

In [ ]:
ultimate = df_messages_enviadas[df_messages_enviadas['plan'] == 'ultimate']
w = (ultimate[ultimate['gb_used_arredondado'] != 0])
w.boxplot('gb_used_arredondado')

In [ ]:
surf = df_messages_enviadas[df_messages_enviadas['plan'] == 'surf']
w = (surf[surf['gb_used_arredondado'] != 0])
w.boxplot('gb_used_arredondado');

In [ ]:
w['gb_used_arredondado'].describe()

Verificando a abertura dos dados de utilização de internet "sem Zero de utilização de dados", identidicamos que a mediana esta em 17GB de utilização, sendo que o plano abrange 15GB, isto significa que grande parte de usuários ultrapassam os pacotes de dados pagando excedentes o que não é vantagem para o cliente, e com o diagrama de caixa temos a comprovação para embasar os dados, onde é visível uma maior quantidade de massa de usuários como outliers acima do limite do pacote grátis do plano e a situação inversa para os usuários do plano ultimate que possuem uma mediana abaixo da utilização do limite do plano de 30GB

Para o plano surf temos uma sugestão seguindo os dados da distribuição de ofertar um upgrade de plano, ou criar um plano intermediário de dados que fique entre 17GB a 20GB, já atenderia a esta demanda de usuários.



[Formule conclusões sobre como os usuários tendem a consumir o tráfego da Internet? O comportamento deles é diferente entre os planos?]

## Receita

[Da mesma forma que você estudou o comportamento do usuário, descreva estatisticamente a receita entre os planos.]

In [ ]:
w = df_messages_enviadas.groupby('plan')['receita_usuario'].mean()
w.plot(title='Receita', kind='pie',figsize=(6,4),autopct='%1.1f%%',colors=['#03fcf0','#0335fc']);
w

Verificamos que no gráfico acima nos mostra que o percentual de média de receita por plano é maior para o ultimate, informando que ele tem mais peso em relação ao surf

In [ ]:
# verificando a somatória pagamento plano mensal ao longo do ano
df_messages_enviadas.pivot_table(index= 'plan',columns= 'mes',values='receita_usuario',aggfunc='sum').plot(kind='bar',figsize=(7,4),alpha=0.5);

In [ ]:
df_messages_enviadas.head()

[Formule conclusões sobre como a receita difere entre os planos.]

## Testar hipóteses estatísticas

[Teste a hipótese de que a receita média dos usuários dos planos de chamadas Ultimate e Surf é diferente.]

[Formule as hipóteses nula e alternativa, escolha o teste estatístico, decida o valor alfa.]

In [ ]:
# Teste as hipóteses
# Receita média dos usuários 'ultimate'

df_messages_enviadas[df_messages_enviadas['plan']=='ultimate']['receita_usuario'].mean()

In [ ]:
# Receita média dos usuários 'surf'

df_messages_enviadas[df_messages_enviadas['plan']=='surf']['receita_usuario'].mean()

In [ ]:
# realizando teste de hipótese BILATERAL

plano_surf = df_messages_enviadas[df_messages_enviadas['plan']=='surf']['receita_usuario']

plano_ultimate = df_messages_enviadas[df_messages_enviadas['plan']=='ultimate']['receita_usuario']

# significância estatística crítica
 
alpha = 0.05  

resultado = st.ttest_ind(plano_surf, plano_ultimate, equal_var=False)

print('p-value: ', resultado.pvalue)

if resultado.pvalue < alpha:
    print('REJEITAMOS A HIPÓTESE NULA')
else:
    print('NÃO REJEITAMOS A HIPÓTESE NULA')    

Ao realizar o teste de hipótese verificando a média das duas amostras planos surf e ultimate, detectamos valores diferentes o que o mesmo se confirma ao verificar suas variâncias, podemos afirmar categoricamente que a receita dos planos são estatisticamente diferentes

Podemos verificar que as receitas entre os planos surf e ultimate possuem médias diferentes o que se encaixa na hipótese alternativa

Iremos no próximo passo verificar a variância embassando com maior propriedade para determinar se as variãncia são iguais ou não

[Teste a hipótese de que a receita média dos usuários da área de NY-NJ difere da dos usuários das demais regiões.]

[Formule as hipóteses nula e alternativa, escolha o teste estatístico, decida o valor alfa.]

In [ ]:
# Teste as hipóteses
# Receita média dos usuários da cidade NY-NJ

df_messages_enviadas[df_messages_enviadas['city']=='New York-Newark-Jersey City, NY-NJ-PA MSA']['receita_usuario'].mean()

In [ ]:
# Receita média dos usuários das outras cidades

df_messages_enviadas[df_messages_enviadas['city']!='New York-Newark-Jersey City, NY-NJ-PA MSA']['receita_usuario'].mean()

In [ ]:
# Variância da receita da cidade 'NY_NJ'

New_York_New_Jersey = df_messages_enviadas[df_messages_enviadas['city']!='New York-Newark-Jersey City, NY-NJ-PA MSA']['receita_usuario']
New_York_New_Jersey.var()

In [ ]:
# Variância da receita das outras cidades

city_other = df_messages_enviadas[df_messages_enviadas['city']=='New York-Newark-Jersey City, NY-NJ-PA MSA']['receita_usuario']
city_other.var()

In [ ]:
# realizando teste de hipótese BILATERAL

New_York_New_Jersey = df_messages_enviadas[df_messages_enviadas['city']!='New York-Newark-Jersey City, NY-NJ-PA MSA']['receita_usuario']

city_other = df_messages_enviadas[df_messages_enviadas['city']=='New York-Newark-Jersey City, NY-NJ-PA MSA']['receita_usuario']

# significância estatística crítica

alpha = 0.05 

resultado = st.ttest_ind(New_York_New_Jersey, city_other, equal_var=True)

print('p-value: ', resultado.pvalue)

if resultado.pvalue < alpha:
    print('REJEITAMOS A HIPÓTESE NULA')
else:
    print('NÃO REJEITAMOS A HIPÓTESE NULA')    

detectamos valores diferentes o que o mesmo se confirma ao verificar suas variâncias, podemos afirmar categoricamente que a receita dos planos são estatisticamente diferentes e assim podemos compreender que há diferenças estatísticas entre as receitas de usuários das cidades de Nova York e New Jersey com as demais cidades.

Conclusão geral

## Conclusão geral

[Liste suas conclusões importantes nessa seção final, certifique-se de que elas cobrem todas as decisões importantes (suposições) que você tomou e que o levaram à maneira como você processou e analisou os dados.]

Foram realizadas tratativas de dados ausentes, enriquecimento de dados, verificação de dados duplicados, construção de visualizações para suportar o embasamento das informações transmitidas dentro deste estudo de caso.

Com base nas informações fornecidas, é possível concluir que a análise dos dados permitiu sugerir políticas de negócios e ações para a empresa com o objetivo de aumentar a fidelização e a experiência do usuário. Além disso, foram realizadas visualizações em gráficos e medidas estatísticas para compreender melhor os comportamentos e consumo dos clientes.

Também foram realizados testes de hipóteses para verificar se a receita média dos usuários dos planos surf e ultimate e se a receita média dos usuários da cidade de Nova York e New Jersey eram iguais às outras cidades. Concluiu-se que as hipóteses nulas foram rejeitadas, o que sugere que há diferenças significativas entre essas receitas médias testadas.

Aqui vai algumas sugestões para a empresa tentando satisfazer mais os clientes

- Ofertar o upgrade para aqueles usuários que mais excedem a utilização dos serviços para o plano ultimate.

- Entre o plano surf e ultimate existe um gap de valores um tem um investimento de 20(dolares) e o outro 70(dolares), valor de mais de um dobro a mais, a empresa tem a oportunidade de criar um 3 plano intermediário que atenda esta parcela de usuários excedentes, por exemplo ofertar uma utilização de 17GB a 20GB já atenderia a gama de excedentes de utilização de internet, o demais raciocínio, pode ser aplicado para os demais serviços mensagens e chamadas, no qual o valor financeiro fique entre a média de 20 a 70.

- Para os dados de chamadas e internet de utilização ZERO (isto é o cliente não usou o serviço ou teve alguma possibilidade de intercorrência técnica), pode se realizar uma pesquisa pós atendimento para verificar a satisfação destes usuários em relação a este serviço, além de checar a possibilidade junto a equipe de infra estrutura a possibilidade de rastreamento de queda de sinal

- Temos duas cidades que se destacaram em ter mais clientes na sua região que foram Nova york - New jersey e Los Angeles, realizar uma beachmarketing com os gestores da localidade para replicar mas mesmas práticas para as demais cidades que se encontram em patamares medianos, o que os gestores de Nova york - New jersey realizam de diferente? que sua clientela é mais que o dobro das demais? saber as diferenças e replicar para os demais

In [ ]:
df.to_csv('arquivo_limpo.csv', index=False)